In [1]:
import os

import pandas as pd
import numpy as np

import hopsworks

from datetime import datetime, timedelta
from pytz import timezone

from src.webscraping import (
    activate_web_driver,
    scrape_to_dataframe,
    convert_columns,
    combine_home_visitor,  
    get_todays_matchups,
)

from src.data_processing import (
    process_games,
    add_TARGET,
)

from src.feature_engineering import (
    process_features,
)

from src.hopsworks_utils import (
    save_feature_names,
    convert_feature_names,
)

import json

from pathlib import Path  #for Windows/Linux compatibility
DATAPATH = Path(r'data')

**Load API keys**

In [2]:
from dotenv import load_dotenv

load_dotenv()

try:
    HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']
except:
    raise Exception('Set environment variable HOPSWORKS_API_KEY')

**Activate Webdriver**

In [3]:
# initiate a webdriver in selenium 
# since website data is dynamically generated

driver = activate_web_driver('firefox')

2023-01-01 10:37:35,566 INFO: ====== WebDriver manager ======
2023-01-01 10:37:35,864 INFO: Get LATEST geckodriver version for 108.0 firefox


[WDM] - Downloading: 19.0kB [00:00, 9.73MB/s]                   


2023-01-01 10:37:36,430 INFO: Driver [C:\Users\Chris\.wdm\drivers\geckodriver\win64\0.32\geckodriver.exe] found in cache


**Scrape New Completed Games and Format Them**

In [4]:

def get_new_games(driver)-> pd.DataFrame:

    # set search for yesterday's games
    DAYS = 1
    SEASON = "" #no season will cause website to default to current season, format is "2022-23"
    TODAY = datetime.now(timezone('EST')) #nba.com uses US Eastern Standard Time
    LASTWEEK = (TODAY - timedelta(days=DAYS))
    DATETO = TODAY.strftime("%m/%d/%y")
    DATEFROM = LASTWEEK.strftime("%m/%d/%y")


    df = scrape_to_dataframe(driver, Season=SEASON, DateFrom=DATEFROM, DateTo=DATETO)

    df = convert_columns(df)

    print(df.info())
    df = combine_home_visitor(df)

    return df

#df_new = get_new_games(driver)

# get the SEASON of the last game in the database
# this will used when constructing rows for prediction
#SEASON = df_new['SEASON'].max()

#df_new




**Retrieve todays games**

In [5]:
#retrieve list of teams playing today

# get today's games on NBA schedule
teams_list, game_id_list = get_todays_matchups(driver)

# example output:
# ['/team/1610612759/spurs/', '/team/1610612748/heat/',...

print(teams_list)
print(game_id_list)

# create list of matchups by parsing out team ids from teams_list
# second team id is always the home team
team_count = len(teams_list) 
matchups = []
for i in range(0,team_count,2):
    visitor_id = teams_list[i].partition("team/")[2].partition("/")[0] #extract team id from text
    home_id = teams_list[i+1].partition("team/")[2].partition("/")[0]
    matchups.append([visitor_id, home_id])

# create list of game IDs by parsing out from game_id_list
games = []
for game in game_id_list:
    game_id = game.partition("-00")[2] #extract team id from text
    if len(game_id) > 0:   #some games have no game id
        games.append(game_id)   

games

<div class="ScheduleDay_sdGames__NGdO5"><div class="ScheduleGame_sg__RmD9I"><div class="ScheduleGame_sgContent__lHCIC"><div class="ScheduleGame_sgStatus__NqtTI"><span class="ScheduleStatusText_base__Jgvjb">8:00 pm ET</span><div class="Broadcasters_base__Wet1u ScheduleGame_sgBroadcasters__O2nwT Broadcasters_vertical__cOo2a" tabindex="0"><div class="Broadcasters_section__ISlyP"><a data-content="SAC @ MEM" data-content-id="0022200547" data-content-type="packages" data-id="nba:schedule:subscribe:undefined" data-section="schedule:broadcasters" data-text="LEAGUE PASS" data-track="click" data-type="logo" href="https://www.nba.com/watch/league-pass-stream"><img alt="LEAGUE PASS" class="Broadcasters_icon__82MTV" role="presentation" src="https://cdn.nba.com/logos/nba/broadcast_logos/L/lp-hor.svg" title="LEAGUE PASS" width="100"/></a></div><div class="Broadcasters_section__ISlyP"><p class="Broadcasters_title__B1dGd">LOCAL TV:</p><p><span class="Broadcasters_tv__AIeZb" data-link-enabled="false">Ba

['22200547', '22200548', '22200549']

**Close Webdriver**

In [6]:
driver.close() 

**Create Blank Rows**

In [7]:
# append today's matchups to the new games dataframe

# since we don't have access to official game ids, we will use the format 20MMDD00 + index as a game id
# This format is used to insure that feature engineering will treat this as a regular season game
# and to be able to find these incomplete games in the database
# this data will be updated later after the game is played

DUMMY_GAME_ID = int("20" + datetime.now(timezone('EST')).strftime("%m%d") + "00")

df_today = df_new.drop(df_new.index) #empty copy of df_new with same columns
for i, matchup in enumerate(matchups):
    game_details = {'HOME_TEAM_ID': matchup[1], 
                    'VISITOR_TEAM_ID': matchup[0], 
                    'GAME_DATE_EST': datetime.now(timezone('EST')).strftime("%Y-%m-%d"), 
                    'GAME_ID': DUMMY_GAME_ID+i,
                    'SEASON': SEASON,
                    } 
    game_details_df = pd.DataFrame(game_details, index=[i])
    # append to new games dataframe
    df_today = pd.concat([df_today, game_details_df], ignore_index = True)

#blank rows will be filled with 0 to prevent issues with feature engineering
df_today = df_today.fillna(0) 

df_today


NameError: name 'df_new' is not defined

**Access Feature Store**

In [ ]:
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()

**Access Feature Group**

In [ ]:
rolling_stats_fg = fs.get_feature_group(
    name="rolling_stats",
    version=1,
)

**Query Old Data Needed for Feature Engineering of New Data**

To generate features like rolling averages for the new games, older data from previous games is needed since some of the rolling averages might extend back 15 or 20 games or so.

In [ ]:
BASE_FEATURES = ['game_date_est',
 'game_id',
 'home_team_id',
 'visitor_team_id',
 'season',
 'pts_home',
 'fg_pct_home',
 'ft_pct_home',
 'fg3_pct_home',
 'ast_home',
 'reb_home',
 'pts_away',
 'fg_pct_away',
 'ft_pct_away',
 'fg3_pct_away',
 'ast_away',
 'reb_away',
 'home_team_wins',
]

ds_query = rolling_stats_fg.select(BASE_FEATURES)
df_old = ds_query.read()
df_old


**Convert Feature Names back to original mixed case**

In [ ]:
df_old = convert_feature_names(df_old)
df_old

**Update Yesterday's Matchup Predictions with New Final Results**

In [ ]:
# filter out games that are pending final results
# (these were the rows used for prediction yesterday)
# and then update these with the new results
#>>> df1.set_index('Code', inplace=True)
#>>> df1.update(df2.set_index('Code'))
#>>> df1.reset_index()  # to recover the initial structure

df_pending = df_old[df_old['GAME_ID'] < 20000100]

df_pending.update(df_new)

df_pending

**Add Today's Matchups for Feature Engineering**

In [ ]:
df_combined = pd.concat([df_combined, df_today], ignore_index = True)
df_combined

**Data Processing**

In [ ]:
df_combined = process_games(df_combined) 
df_combined = add_TARGET(df_combined)
df_combined

**Feature Engineering**

In [ ]:
# Feature engineering to add: 
    # rolling averages of key stats, 
    # win/lose streaks, 
    # home/away streaks, 
    # specific matchup (team X vs team Y) rolling averages and streaks

df_combined = process_features(df_combined)
df_combined


**Insert New Data into Feature Group**

In [ ]:

def test():
    # retrieve only new games from the combined dataframe now that feature engineering is complete

    # set index to GAME_ID
    df_combined = df_combined.set_index('GAME_ID')
    df_new = df_new.set_index('GAME_ID')
    
    # retrieve only new games
    df_new = df_combined.loc[df_new.index]

    # reset GAME_ID index back to column
    df_new = df_new.reset_index()

    # convert certain features back to int32 for Hopsworks compatibility
    df_new['GAME_ID'] = df_new['GAME_ID'].astype('int32')
    df_new['HOME_TEAM_WINS'] = df_new['HOME_TEAM_WINS'].astype('int32')
    df_new['TARGET'] = df_new['TARGET'].astype('int32')

    # save new games to Hopsworks feature group
    rolling_stats_fg.insert(df_new, write_options={"wait_for_job" : False})

    df_new 

rolling_stats_fg.insert(df_combined, write_options={"wait_for_job" : False})